In [ ]:
#requirments
!pip install diffusers
!pip install torch
!pip install accelerate
import scipy
import os
# from diffusers import AudioLDM2Pipeline
# from diffusers.models import AutoencoderKL
# from diffusers import StableDiffusionPipeline
# from transformers import SpeechT5HifiGan
import torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
PATH = "/content/drive/My Drive"
from accelerate import Accelerator


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
path_to_model = "/content/drive/My Drive/final_unet"
# from diffusers.models import AutoencoderKL
if torch.cuda.is_available():
  p_device = ('cuda')
else:
  p_device = ('cpu')

vae = torch.load("/content/drive/My Drive/vae",map_location=p_device,weights_only=False).eval()
vocoder = torch.load("/content/drive/My Drive/vocoder",map_location=p_device).eval()
scheduler = torch.load("/content/drive/My Drive/scheduler")
unet = torch.load("/content/drive/My Drive/final_unet").eval().to(p_device)
accelerator = Accelerator()
vae, vocoder, scheduler, unet = accelerator.prepare(vae, vocoder, scheduler, unet)

models = {
    "vae":vae,
    "vocoder":vocoder,
    "scheduler":scheduler,
    "unet": unet
    }


In [ ]:
tensor = torch.randn((1,1,1024,64),dtype=torch.float16,device = p_device)
latents = models['vae'].encode(tensor).latent_dist.sample()* 0.18215
latents = latents.type(torch.float32)
latents.dtype
# print(encoded.shape)

torch.float32

In [ ]:
n = torch.randn((1,64,1024),dtype=torch.float32,device = p_device)
# n1 = torch.randn_like(encoded,dtype=torch.float32,device = p_device)
# diff_out = models['unet'](encoded.type(torch.float32),timestep = 1, encoder_hidden_states= n,return_dict=False)[0]

In [ ]:
import inspect
def prepare_extra_step_kwargs(generator, eta):
    # prepare extra kwargs for the scheduler step, since not all schedulers have the same signature
    # eta (η) is only used with the DDIMScheduler, it will be ignored for other schedulers.
    # eta corresponds to η in DDIM paper: https://arxiv.org/abs/2010.02502
    # and should be between [0, 1]

    accepts_eta = "eta" in set(inspect.signature(models['scheduler'].step).parameters.keys())
    extra_step_kwargs = {}
    if accepts_eta:
        extra_step_kwargs["eta"] = eta

    # check if the scheduler accepts generator
    accepts_generator = "generator" in set(inspect.signature(models['scheduler'].step).parameters.keys())
    if accepts_generator:
        extra_step_kwargs["generator"] = generator
    return extra_step_kwargs
eta = 0.0
generator = torch.Generator(device='cuda')
extra_step_kwargs = prepare_extra_step_kwargs(generator , eta)

In [ ]:
num_inference_steps =200
scheduler.set_timesteps(num_inference_steps, 'cuda')
timesteps = scheduler.timesteps
num_warmup_steps = len(timesteps) - num_inference_steps * scheduler.order

for i, t in enumerate(timesteps):
  print(i)
  # expand the latents if we are doing classifier free guidance
  # latent_model_input = torch.cat([latents] * 2) if do_classifier_free_guidance else latents
  latent_model_input = latents
  latent_model_input = scheduler.scale_model_input(latent_model_input, t)

  # predict the noise residual
  with torch.no_grad():
    noise_pred = unet(latents,timestep = t, encoder_hidden_states= n,return_dict=False)[0]
  latents = scheduler.step(noise_pred, t, latents, **extra_step_kwargs).prev_sample

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199


In [ ]:
# log_mels = models['vae'].decode(diff_out.type(torch.half))
log_mels = models['vae'].decode(latents.type(torch.float16))
print(log_mels.sample.shape)

torch.Size([1, 1, 1024, 64])


In [ ]:
# Copied from diffusers.pipelines.audioldm.pipeline_audioldm.AudioLDMPipeline.mel_spectrogram_to_waveform
def mel_spectrogram_to_waveform(mel_spectrogram):
  if mel_spectrogram.dim() == 4:
      mel_spectrogram = mel_spectrogram.squeeze(1)
      print("was here")

  waveform = models['vocoder'](mel_spectrogram)
  # we always cast to float32 as this does not cause significant overhead and is compatible with bfloat16
  waveform = waveform.cpu().float()
  return waveform

In [ ]:
from IPython.display import Audio
original_waveform_length = 163872
# n = torch.randn((7,64,64),dtype=torch.float16).cuda()
# print(n.squeeze(0).shape)
audio = mel_spectrogram_to_waveform(log_mels.sample)
print(audio.shape)
# audio = mel_spectrogram_to_waveform()
audio = audio[:, :original_waveform_length].detach().numpy()
Audio(audio, rate=16000)

was here
torch.Size([1, 163872])
